In [ ]:
import sys
import numpy as np
import pygame, random
from pygame.locals import *

#POSICAO ALEATORIA
def on_grid_random(): 
    x = random.randint(BORDA+10,X_SCREEN-BORDA-10)
    y = random.randint(BORDA_SUP+10,Y_SCREEN-BORDA_INF-10)
    return (x//10 * 10, y//10 * 10)

#SE COMER A MAÇÃ
def eat(c1,c2): 
    return (c1[0] == c2[0]) and (c1[1] == c2[1])

#SE BATER NA PAREDE
def collision(snake, A): 
    return ((snake[0]<=BORDA) or (snake[0]>=X_SCREEN-BORDA-10)) or ((snake[1]<=BORDA_SUP) or (snake[1]>=Y_SCREEN-BORDA_INF-10))


#CONSTANTES
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
X_SCREEN = 500
Y_SCREEN = 800
A = (X_SCREEN,Y_SCREEN)
BORDA_SUP = 100
BORDA_INF = 50
BORDA = 10

#VARIAVEIS
SPEED = 15
speed = SPEED
SCORE = 0
LAST_SCORE = 0
MODE = 'START'

#COBRA
snake_pos = on_grid_random()
snake = [snake_pos,(snake_pos[0]+10,snake_pos[1]),(snake_pos[0]+20,snake_pos[1])]
snake_skin = pygame.Surface((10,10))
snake_skin.fill((0,255,0))

#MAÇÃ
apple_pos = on_grid_random()
apple = pygame.Surface((10,10))
apple.fill((255,0,0))

#PAREDE
wall_pos = (10,100)
wall = []
for x in range(BORDA,X_SCREEN-BORDA,10):
    wall.append((x,BORDA_SUP))
    wall.append((x,Y_SCREEN-BORDA_INF-10))
for y in range(BORDA_SUP,Y_SCREEN-BORDA_INF,10):
    wall.append((BORDA,y))
    wall.append((X_SCREEN-BORDA-10,y))
wall_skin = pygame.Surface((10,10))
wall_skin.fill((255,255,255))

#CONFIG JOGO
pygame.init()
screen = pygame.display.set_mode(A)
pygame.display.set_caption('Snake Game')
myfont = pygame.font.SysFont("monospace", 20)
scorefont = pygame.font.SysFont("monospace", 50)
statusfont = pygame.font.SysFont("arial", 60)
status2font = pygame.font.SysFont("arial", 20)
my_direction = LEFT
clock = pygame.time.Clock()

while True:
    clock.tick(speed) #VELOCIDADE DA COBRA
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
        if event.type == KEYDOWN: #COMANDOS PARA CORRER
            if event.key == K_UP and my_direction != DOWN:
                my_direction = UP
            if event.key == K_DOWN and my_direction != UP:
                my_direction = DOWN
            if event.key == K_LEFT and my_direction != RIGHT:
                my_direction = LEFT
            if event.key == K_RIGHT and my_direction != LEFT:
                my_direction = RIGHT
            if event.key == K_PAUSE:
                MODE = 'STOP'
            if event.key == K_SPACE:
                MODE = 'PLAY'
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
                
    screen.fill((0,0,0)) #COR FUNDO
    screen.blit(apple,apple_pos) #POSIÇÃO MAÇÃ
    
    #SE A COBRA BATER NA PAREDE
    if collision(snake[0],A): 
        snake_pos = on_grid_random()
        snake = [snake_pos,(snake_pos[0]+10,snake_pos[1]),(snake_pos[0]+20,snake_pos[1])]
        apple_pos =  on_grid_random()
        MODE = 'END'
        speed = SPEED
        LAST_SCORE = SCORE
        SCORE = 0
    
    #SE A COBRA COMER A MAÇA
    if eat(snake[0], apple_pos):
        apple_pos =  on_grid_random()
        snake.append((0,0))
        speed+=1
        SCORE+=1
        
    #PLAY STATUS   
    if MODE == 'PLAY':    
        for i in range(len(snake) - 1, 0, -1):
            snake[i] = (snake[i-1][0], snake[i-1][1])
        if my_direction == UP:
            snake[0] = (snake[0][0],snake[0][1] - 10)
        if my_direction == DOWN:
            snake[0] = (snake[0][0],snake[0][1] + 10)
        if my_direction == RIGHT:
            snake[0] = (snake[0][0] + 10,snake[0][1])
        if my_direction == LEFT:
            snake[0] = (snake[0][0] - 10,snake[0][1])
            
    #START STATUS
    if MODE == 'START':
        statustext = statusfont.render("SNAKE GAME", 1, (0,255,0))
        screen.blit(statustext, ((X_SCREEN/2)-200, Y_SCREEN/2))
        status2text = status2font.render("Press space to start..", 1, (255,255,255))
        screen.blit(status2text, ((X_SCREEN/2)-120, (Y_SCREEN/2)+70))
    
    #STOP STATUS
    if MODE == 'STOP':
        statustext = statusfont.render("PAUSED", 1, (255,0,0))
        screen.blit(statustext, ((X_SCREEN/2)-130, Y_SCREEN/2))
        status2text = status2font.render("Press space to continue..", 1, (255,150,150))
        screen.blit(status2text, ((X_SCREEN/2)-120, (Y_SCREEN/2)+70))
    
    #END STATUS
    if MODE == 'END':
        lastscoretext = status2font.render("YOUR SCORE: {}".format(LAST_SCORE), 1, (150,150,255))
        screen.blit(lastscoretext, ((X_SCREEN/2)-170, (Y_SCREEN/2)-70))
        statustext = statusfont.render("GAME OVER", 1, (255,0,0))
        screen.blit(statustext, ((X_SCREEN/2)-170, (Y_SCREEN/2)))
        status2text = status2font.render("Press space to continue..", 1, (255,150,150))
        screen.blit(status2text, ((X_SCREEN/2)-120, (Y_SCREEN/2)+70))

    disclaimertext = myfont.render("rodolfoshimotsu@gmail.com", 1, (150,150,255))
    screen.blit(disclaimertext, ((X_SCREEN/2)-150, Y_SCREEN-40))

    scoretext = scorefont.render("SCORE: {0}".format(SCORE), 1, (150,150,255))
    screen.blit(scoretext, (20, 30))

    for pos in snake:
        screen.blit(snake_skin,pos)
    for w in wall:
        screen.blit(wall_skin,w)
        
    pygame.display.update()